In [1]:
!pip install newsapi-python
!pip install newspaper3k

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.1/211.1 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.6/97.6 kB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 15.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for tinysegmenter: filename=tinysegmenter-0.3-py3-none-any.whl size=13538 sha256=f6773a4ed4239b6baa1b6e12da2e84bdb9d163ab2a4bee4aee1cb73d1db6be95
  Stored in directory: /root/.cache/pip/wheels/c8/d6/6c/384f58df48c00b9a31d638005143b5b3ac62c3d25fb1447f23
  Created wheel for feedfinder2: filename=feedfinder2-0.0.4-py3-none-any.whl size=3340 sha256=e1bfe59577acfb18149f1ff3dd2d9d19709529145cd1c7a75f60ae9851b873c7
  Stored in directory: /root/.cache/pip/wheels/97/02/e7/a1ff1760e12bdbaab0ac824fae5c1bc

In [2]:
#set up
from newsapi import NewsApiClient
from newspaper import fulltext
import requests
from requests import ReadTimeout, ConnectTimeout, HTTPError, Timeout, ConnectionError
import pandas as pd

newsapi = NewsApiClient(api_key='53f9ca03b7c14c858c2f072fe04b180c')
sources = '''abc-news,bloomberg,business-insider,cbs-news,cnn,fortune,fox-news,google-news,medical-news-today,msnbc,
    mtv-news,nbc-news,new-york-magazine,techcrunch,the-huffington-post,the-wall-street-journal,the-washington-times,
    time,usa-today,wired'''

In [3]:
cur_date = '2024-04-01'
page = 1

In [4]:
data = []

def add_new_articles(articles):
    for a in articles:
        try:
            r = requests.get(a['url'], timeout=3.0)
            ad = {
                'content': fulltext(r.text),
                'summary': a['description']
            }
            if ad['summary'] == 'NaN':
              break
            data.append(ad)
        except (ConnectTimeout, HTTPError, ReadTimeout, Timeout, ConnectionError):
            print(a['url'])

all_articles = newsapi.get_everything(
                                      sources=sources,
                                      from_param=cur_date,
                                      to=cur_date,
                                      language='en',
                                      sort_by='popularity',
                                      page=1)
tr = all_articles['totalResults']
add_new_articles(all_articles['articles'])
page = 1
while page*100 < tr:
    page += 1
    print(page)
    all_articles = newsapi.get_everything(
                                      sources=sources,
                                      from_param=cur_date,
                                      to=cur_date,
                                      language='en',
                                      sort_by='popularity',
                                      page=page)
    add_new_articles(all_articles['articles'])

https://disneyparks.disney.go.com/blog/2024/04/celebrate-earth-month-with-new-disney-wallpapers-magic-shots/
2
3
4
https://abcnews.go.com/US/wireStory/trumps-immigration-rhetoric-makes-inroads-democrats-concern-biden-108702095
https://abcnews.go.com/US/wireStory/gaza-medical-officials-apparent-israeli-airstrike-killed-4-108724166
5
6


NewsAPIException: {'status': 'error', 'code': 'maximumResultsReached', 'message': 'You have requested too many results. Developer accounts are limited to a max of 100 results. You are trying to request results 100 to 120. Please upgrade to a paid plan if you need more results.'}

In [5]:
df = pd.DataFrame(data)
df = df.dropna(subset=['summary'])
df = df[df['content'].apply(lambda x: len(x) >= 500)]

In [6]:
print(len(df))

421


In [ ]:

df.to_csv(f'{cur_date}.csv',index=False)